# Creating Haiku

This notebook will focus on actually generating haiku using the prompts previously created. This notebook/process is actually the most simple part of the project, but I'll point out a few place where changes could be made to the approach. 

As before we'll install `distilable` with the `vllm` extra. 

In [ ]:
%pip install distilabel['vllm']

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 82.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 97.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

We'll start by importing the necessary libraries

In [ ]:
from datasets import load_dataset
from distilabel.llm import vLLM
from distilabel.pipeline import Pipeline
from distilabel.tasks import TextGenerationTask
from vllm import LLM

We'll load the prompts dataset which we previously created from the Hugging Face Hub. 

In [ ]:
prompts = load_dataset("davanstrien/haiku_prompts",split="train")
dataset = prompts.rename_column("instructions", "input")
dataset

Generating train split:   0%|          | 0/4303 [00:00<?, ? examples/s]

Dataset({
    features: ['input'],
    num_rows: 4303
})

## Creating our LLM generator

We'll now create the LLM we'll use to generate our haiku. We'll create a `TextGenerationTask` in distilabel. We can also pass in a `system_prompt`, in this case we'll prompt the model to focus on the "technical" structure of a haiku whilst also encouraging the model to be "creative".

In [ ]:
task = TextGenerationTask(
    system_prompt="""You are a poet specialising in creating Haiku. \nYour haiku consist of three lines, with five syllables in the first line, seven in the second, and five in the third.\nBeyond being technically correct, your haiku should also be beautiful and meaningful"""
)

In [ ]:
print(task.system_prompt)

You are a poet specialising in creating Haiku. 
Your haiku consist of three lines, with five syllables in the first line, seven in the second, and five in the third.
Beyond being technically correct, your haiku should also be beautiful and meaningful


Very similar to the previous notebook we'll wrap a `vLLM`  LLM in a distilabel weraper, pass in our task, `max_new_tokens` (which we can keep pretty short in this case), and set a temperature of 0.7. This will allow the model to be more "creative". 

In [ ]:
generator = vLLM(
    vllm=LLM(model="TheBloke/OpenHermes-2.5-Mistral-7B-AWQ"),
    task=task,
    max_new_tokens=128,
    temperature=0.7,
    prompt_format="chatml",
)

config.json:   0%|          | 0.00/837 [00:00<?, ?B/s]

WARNING 01-14 19:06:53 config.py:175] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 01-14 19:06:53 llm_engine.py:70] Initializing an LLM engine with config: model='TheBloke/OpenHermes-2.5-Mistral-7B-AWQ', tokenizer='TheBloke/OpenHermes-2.5-Mistral-7B-AWQ', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=awq, enforce_eager=False, seed=0)


tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/101 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors:   0%|          | 0.00/4.15G [00:00<?, ?B/s]

INFO 01-14 19:10:42 llm_engine.py:275] # GPU blocks: 4577, # CPU blocks: 2048
INFO 01-14 19:10:44 model_runner.py:501] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 01-14 19:10:44 model_runner.py:505] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode.
INFO 01-14 19:10:57 model_runner.py:547] Graph capturing finished in 13 secs.


As before, we now create a `Pipeline` which takes in our generator. 

In [ ]:
pipeline = Pipeline(generator=generator)

As before, we can now call the `generate` method on our pipeline and pass in our prompt dataset. We also specify that we want `20` generations for each prompt. This might be overkill but will hopefully give us more diverse data for the next part of this project where we begin to evaluate our generated haiku.

In [ ]:
haikus = pipeline.generate(
    dataset, num_generations=20, batch_size=4, display_progress_bar=True
)


Texts Generated ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 100% 86000/86060
Rows labelled   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0%    0/4303  

Flattening the indices:   0%|          | 0/4303 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4303 [00:00<?, ? examples/s]

INFO:distilabel:Final dataset saved at /content/ckpt


In [ ]:
haikus.push_to_hub("davanstrien/haiku_dpo", "raw-haikus", private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/9.03k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/davanstrien/haiku_dpo/commit/db03db2a9b69d6a5f2ca544d4621d5ee546ae87f', commit_message='Upload dataset', commit_description='', oid='db03db2a9b69d6a5f2ca544d4621d5ee546ae87f', pr_url=None, pr_revision=None, pr_num=None)